In [3]:
#!pip install yfinance

In [2]:
import yfinance as yf #to get stock prices from internet
import numpy as np #for calcs
import pandas as pd #data handling
import datetime as dt
from datetime import datetime
from pandas.tseries.offsets import MonthEnd #to jump through months 

In [5]:
# Import all ticker symbols from s&p500
snp500_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0] #pull all tables from website and read first item in list of tables

In [6]:
snp500_tickers = snp500_df['Symbol'].to_list()

In [8]:
#get stocks prices for all stocks
date_today = datetime.now()
snp500_stock_info_df = yf.download(snp500_tickers, start='2020-1-31',end=date_today)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
[*********************102%************************]  514 of 505 completed

In [10]:
#get adjusted close prices only 
snp500_stock_prices = snp500_stock_info_df['Adj Close']
snp500_stock_prices.index = pd.to_datetime(snp500_stock_prices.index) #convert string date index to date format

In [16]:
#calc monthly returns
snp500_monthly_return_df = snp500_stock_prices.pct_change().resample('M').agg(lambda x: (x+1).prod()-1) #get daily change and accum for a month

In [19]:
#calc returns over 11 month window
snp500_rolling_11_month_window_df = (snp500_monthly_return_df+1).rolling(11).apply(np.prod)-1

In [21]:
#porfolio review buying winners and selling losers
formation = dt.datetime(2020,12,31)
end_measurement = formation - MonthEnd(1) #1 month before formation date

In [28]:
snp500_12month_return = snp500_rolling_11_month_window_df.loc[end_measurement] # search for end measurement date in rolling 11 month data

In [29]:
snp500_12month_return = snp500_12month_return.reset_index()
snp500_12month_return['decile'] = pd.qcut(snp500_12month_return.iloc[:,1],10,labels=False, duplicates='drop') #add new column decile and sort all in column 2 and sort into top 10 categories

In [30]:
#define winners and losers
snp500_winners = snp500_12month_return[snp500_12month_return.decile == 9]
snp500_losers = snp500_12month_return[snp500_12month_return.decile == 0]

In [33]:
#calc estimated winners return and losers return from buying and selling on formation date and holding for 1 month
snp500_winners_return = snp500_monthly_return_df.loc[formation + MonthEnd(1), snp500_monthly_return_df.columns.isin(snp500_winners['index'])] #search for the formation date + 1 month and only for those that match ticker symbols stored in column 'index' for winners
snp500_losers_return = snp500_monthly_return_df.loc[formation + MonthEnd(1), snp500_monthly_return_df.columns.isin(snp500_losers['index'])]

snp500_momentum_profit = snp500_winners_return.mean() - snp500_losers_return.mean()
print('PL 1 month after formation date would be: ',snp500_momentum_profit)

0.002797584986879839

In [ ]:
#https://www.youtube.com/watch?v=5W_Lpz1ZuTI
# 19:25

def snp500_momentum(formation_date):
    end_measurement = formation - MonthEnd(1) #1 month before formation date
